# 複数のベクトルを1つのドキュメントに紐付けて検索する方法

1つのドキュメントに対して複数のベクトルを保存することは、多くの場面で有用です。これにはいくつかの利点や活用例があります。  
例えば、ドキュメントの複数のチャンクを埋め込みとして生成し、それらの埋め込みを親ドキュメントに関連付けることで  
リトリーバーがチャンクに一致した場合でも、親ドキュメント全体を返すことが可能になります。

LangChainでは、このプロセスを簡略化するためにベースクラスとして`MultiVectorRetriever`を実装しています。  
複数のベクトルを1つのドキュメントに紐付ける方法に多くの複雑さが伴いますが、このクラスがそれをシンプルにしてくれます。  
このノートブックでは、一般的なベクトル生成方法と`MultiVectorRetriever`の使用方法について解説します。

複数のベクトルを生成する方法
1. 小さなチャンク
    - ドキュメントを小さなチャンクに分割し、それぞれを埋め込みとして生成します。
    - これは`ParentDocumentRetriever`が行う方法です。
2. 要約
    - 各ドキュメントの要約を作成し、それを埋め込みとして生成します（ドキュメントそのものの埋め込みと併用するか、代わりに利用）。
3. 仮想的な質問
    - 各ドキュメントが適切に回答できる仮想的な質問を作成し、それを埋め込みとして生成します（ドキュメントそのものの埋め込みと併用するか、代わりに利用）。

また、埋め込みを手動で追加する方法もサポートされています。  
これにより、特定の質問やクエリがドキュメントを返すように明示的に設定できるため、検索の制御がより柔軟になります。

以下で、実例を通じて解説します。  
まずいくつかのドキュメントを作成し、それらをOpenAI埋め込みを使用してインメモリのChromaベクトルストアにインデックス化します。  
ただし、他のLangChainベクトルストアや埋め込みモデルを使用することも可能です。

In [ ]:
%pip install --upgrade --quiet  langchain-chroma langchain langchain-openai > /dev/null

In [1]:
# 必要なモジュールをインポート
from langchain.storage import InMemoryByteStore  # メモリ内でのストレージ管理
from langchain_chroma import Chroma  # Chroma ベクトルストアのインターフェース
from langchain_community.document_loaders import TextLoader  # テキストファイルをロードするためのローダー
from langchain_openai import OpenAIEmbeddings  # OpenAIの埋め込みモデルを利用
from langchain_text_splitters import RecursiveCharacterTextSplitter  # ドキュメント分割ツール

# 読み込むテキストファイルを指定し、ローダーを作成
loaders = [
    TextLoader("paul_graham_essay.txt"),  # Paul Graham のエッセイ
    TextLoader("state_of_the_union.txt"),  # 一般教書演説の内容
]

# ドキュメントを格納するリストを初期化
docs = []

# 各ローダーからテキストを読み込み、docsリストに追加
for loader in loaders:
    docs.extend(loader.load())

# 大きなチャンクに分割するためのテキストスプリッターを作成
# chunk_size=10000 で、10,000文字ごとに分割
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000)

# 読み込んだドキュメントをチャンクに分割
docs = text_splitter.split_documents(docs)

# 子チャンクをインデックス化するために使用するベクトルストアを作成
# - "full_documents" というコレクション名で管理
# - OpenAI の埋め込み関数を指定
vectorstore = Chroma(
    collection_name="full_documents",  # コレクションの名前
    embedding_function=OpenAIEmbeddings()  # ベクトル生成に使用する埋め込みモデル
)

## 小さなチャンク

多くの場合、大きな情報のチャンクを取得するのが有用ですが、より小さなチャンクを埋め込むことが役立つことがあります。  
これにより、埋め込みが意味論的な意味をできるだけ正確に捉える一方で、できる限り多くのコンテキストが下流に渡されます。  
これは `ParentDocumentRetriever` が行っている処理です。ここでは、その仕組みの詳細を説明します。

ベクトルストアとドキュメントストアの区別を行います：

ベクトルストア: （サブ）ドキュメントの埋め込みをインデックス化します。  
ドキュメントストア: 「親」ドキュメントを保持し、それらに識別子を関連付けます。

In [2]:
import uuid

from langchain.retrievers.multi_vector import MultiVectorRetriever

# 親ドキュメントを保存するためのストレージ層
store = InMemoryByteStore()
id_key = "doc_id"

# リトリーバー（最初は空の状態で作成）
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,  # ベクトルストア（子チャンクの埋め込みを格納）
    byte_store=store,         # バイトストア（親ドキュメントを格納）
    id_key=id_key,            # ドキュメント識別子のキー
)

# 各ドキュメントに一意のIDを生成
doc_ids = [str(uuid.uuid4()) for _ in docs]


次に、元のドキュメントを分割して「サブ」ドキュメントを生成します。  
この際、対応する `Document` オブジェクトの`metadata`にドキュメント識別子を格納します。

In [3]:
# 小さなチャンクを作成するために使用するスプリッター
child_text_splitter = RecursiveCharacterTextSplitter(chunk_size=400)

sub_docs = []
for i, doc in enumerate(docs):
    _id = doc_ids[i]  # ドキュメント識別子を取得
    _sub_docs = child_text_splitter.split_documents([doc])  # 元のドキュメントを分割
    for _doc in _sub_docs:
        _doc.metadata[id_key] = _id  # メタデータにドキュメント識別子を追加
    sub_docs.extend(_sub_docs)  # サブドキュメントをリストに追加


最後に、ドキュメントをベクトルストアとドキュメントストアにインデックス化します。

In [4]:
retriever.vectorstore.add_documents(sub_docs)  # サブドキュメントをベクトルストアに追加
retriever.docstore.mset(list(zip(doc_ids, docs)))  # 親ドキュメントをドキュメントストアに格納

ベクトルストアだけを使用すると、小さなチャンクが取得されます：

In [5]:
retriever.vectorstore.similarity_search("justice breyer")[0]

Document(page_content='Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court.', metadata={'doc_id': '064eca46-a4c4-4789-8e3b-583f9597e54f', 'source': 'state_of_the_union.txt'})

一方で、リトリーバーはより大きな親ドキュメントを返します：

In [6]:
len(retriever.invoke("justice breyer")[0].page_content)

9875

リトリーバーがベクトルデータベースでデフォルトで行う検索タイプは「類似性検索（similarity search）」です。  
LangChainのベクトルストアは、「最大マージナル関連性（Max Marginal Relevance）」による検索もサポートしています。  
この検索タイプは、リトリーバーのsearch_typeパラメータを使用して制御できます。

In [7]:
from langchain.retrievers.multi_vector import SearchType

retriever.search_type = SearchType.mmr

len(retriever.invoke("justice breyer")[0].page_content)

9875

## ドキュメントに要約を関連付けて検索

要約は、チャンクが何について書かれているかをより正確に抽出するのに役立ち、より良い検索結果をもたらす可能性があります。  
ここでは、要約を作成し、それらを埋め込みに利用する方法を説明します。

ドキュメントオブジェクトを受け取り、LLMを使用して要約を生成するシンプルなチェーンを構築します。

In [9]:
from langchain_openai import ChatOpenAI

# LLMモデルをインスタンス化
llm = ChatOpenAI()

import uuid

from langchain_core.documents import Document  # ドキュメントオブジェクトを扱うためのモジュール
from langchain_core.output_parsers import StrOutputParser  # 出力を文字列に変換するためのパーサー
from langchain_core.prompts import ChatPromptTemplate  # プロンプトテンプレートを扱うためのモジュール

# 要約を生成するためのチェーンを構築
chain = (
    {"doc": lambda x: x.page_content}  # ドキュメントのページ内容を抽出
    | ChatPromptTemplate.from_template("Summarize the following document:\n\n{doc}")  # プロンプトテンプレートを設定
    | llm  # LLMに渡して要約を生成
    | StrOutputParser()  # 出力を文字列として解析
)

#### ドキュメント全体に対するバッチ処理
このチェーンを複数のドキュメントに対して並列で処理できます。

In [10]:
summaries = chain.batch(docs, {"max_concurrency": 5})

次に、前と同様に `MultiVectorRetriever` を初期化し、要約をベクトルストアにインデックス化し、元のドキュメントをドキュメントストアに保持します：

In [11]:
# 子チャンクをインデックス化するためのベクトルストア
vectorstore = Chroma(collection_name="summaries", embedding_function=OpenAIEmbeddings())
# 親ドキュメントを保存するためのストレージ層
store = InMemoryByteStore()
id_key = "doc_id"
# リトリーバー（最初は空の状態で作成）
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,  # ベクトルストア
    byte_store=store,         # ストレージ層
    id_key=id_key,            # ドキュメント識別子
)

# 各ドキュメントに一意のIDを生成
doc_ids = [str(uuid.uuid4()) for _ in docs]

# 要約をドキュメントとして作成し、メタデータにIDを追加
summary_docs = [
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(summaries)
]

# 要約ドキュメントをベクトルストアに追加
retriever.vectorstore.add_documents(summary_docs)
# 元のドキュメントをドキュメントストアに保存
retriever.docstore.mset(list(zip(doc_ids, docs)))

In [17]:
# 必要に応じて、元のチャンクをベクトルストアに追加することも可能
# for i, doc in enumerate(docs):
#     doc.metadata[id_key] = doc_ids[i]
# retriever.vectorstore.add_documents(docs)

ベクトルストアにクエリを実行すると、要約が返されます：

In [12]:
sub_docs = retriever.vectorstore.similarity_search("justice breyer")

sub_docs[0]

Document(page_content="President Biden recently nominated Judge Ketanji Brown Jackson to serve on the United States Supreme Court, emphasizing her qualifications and broad support. The President also outlined a plan to secure the border, fix the immigration system, protect women's rights, support LGBTQ+ Americans, and advance mental health services. He highlighted the importance of bipartisan unity in passing legislation, such as the Violence Against Women Act. The President also addressed supporting veterans, particularly those impacted by exposure to burn pits, and announced plans to expand benefits for veterans with respiratory cancers. Additionally, he proposed a plan to end cancer as we know it through the Cancer Moonshot initiative. President Biden expressed optimism about the future of America and emphasized the strength of the American people in overcoming challenges.", metadata={'doc_id': '84015b1b-980e-400a-94d8-cf95d7e079bd'})

一方、リトリーバーにクエリを実行すると、元の大きなドキュメントが返されます：

In [13]:
retrieved_docs = retriever.invoke("justice breyer")

len(retrieved_docs[0].page_content)

9194

## 仮想的な質問

LLMを使用して、特定のドキュメントに対して質問される可能性のある仮想的な質問のリストを生成することもできます。  
これらの質問は、RAG（Retrieval-Augmented Generation）アプリケーションにおける関連するクエリと意味的に近い関係を持つ可能性があります。
生成された質問は埋め込まれ、ドキュメントに関連付けられることで、検索性能を向上させることができます。  

以下の例では、with_structured_output メソッドを使用して、LLMの出力を文字列リストとして構造化します。

In [16]:
from typing import List

from pydantic import BaseModel, Field

class HypotheticalQuestions(BaseModel):
    """仮想的な質問を生成するクラス"""

    questions: List[str] = Field(..., description="質問のリスト")

chain = (
    {"doc": lambda x: x.page_content}  # ドキュメントの内容を抽出
    # 仮想的な質問を3つ生成するよう指示（必要に応じて調整可能）
    | ChatPromptTemplate.from_template(
        "Generate a list of exactly 3 hypothetical questions that the below document could be used to answer:\n\n{doc}"
    )
    | ChatOpenAI(max_retries=0, model="gpt-4o").with_structured_output(
        HypotheticalQuestions  # 出力を構造化して仮想的な質問のリストを生成
    )
    | (lambda x: x.questions)  # 質問のリストを抽出
)

チェーンを単一のドキュメントに対して呼び出すと、質問のリストが出力されることを確認できます：

In [17]:
chain.invoke(docs[0])

["What impact did the IBM 1401 have on the author's early programming experiences?",
 "How did the transition from using the IBM 1401 to microcomputers influence the author's programming journey?",
 "What role did Lisp play in shaping the author's understanding and approach to AI?"]

次に、このチェーンをすべてのドキュメントに対してバッチ処理し、以前と同様にベクトルストアとドキュメントストアを組み立てます：

In [18]:
# ドキュメントに対してチェーンをバッチ処理し、仮想的な質問を生成
hypothetical_questions = chain.batch(docs, {"max_concurrency": 5})

# 子チャンクをインデックス化するためのベクトルストア
vectorstore = Chroma(
    collection_name="hypo-questions", embedding_function=OpenAIEmbeddings()
)
# 親ドキュメントを保存するためのストレージ層
store = InMemoryByteStore()
id_key = "doc_id"
# リトリーバー（最初は空の状態で作成）
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)

# 各ドキュメントに一意のIDを生成
doc_ids = [str(uuid.uuid4()) for _ in docs]

# 仮想的な質問を用いてDocumentオブジェクトを生成
question_docs = []
for i, question_list in enumerate(hypothetical_questions):
    question_docs.extend(
        [Document(page_content=s, metadata={id_key: doc_ids[i]}) for s in question_list]
    )

# ベクトルストアに仮想的な質問を追加
retriever.vectorstore.add_documents(question_docs)
# ドキュメントストアに元のドキュメントを保存
retriever.docstore.mset(list(zip(doc_ids, docs)))

ベクトルストアにクエリを実行すると、入力クエリと意味的に類似した仮想的な質問が取得されます：

In [19]:
sub_docs = retriever.vectorstore.similarity_search("justice breyer")

sub_docs

[Document(page_content='What might be the potential benefits of nominating Circuit Court of Appeals Judge Ketanji Brown Jackson to the United States Supreme Court?', metadata={'doc_id': '43292b74-d1b8-4200-8a8b-ea0cb57fbcdb'}),
 Document(page_content='How might the Bipartisan Infrastructure Law impact the economic competition between the U.S. and China?', metadata={'doc_id': '66174780-d00c-4166-9791-f0069846e734'}),
 Document(page_content='What factors led to the creation of Y Combinator?', metadata={'doc_id': '72003c4e-4cc9-4f09-a787-0b541a65b38c'}),
 Document(page_content='How did the ability to publish essays online change the landscape for writers and thinkers?', metadata={'doc_id': 'e8d2c648-f245-4bcc-b8d3-14e64a164b64'})]

一方、リトリーバーにクエリを実行すると、対応する親ドキュメントが返されます：

In [20]:
retrieved_docs = retriever.invoke("justice breyer")
len(retrieved_docs[0].page_content)

9194